# Data Preprocessing

In [10]:
import cv2
import os

def flip_image(input_image, output_image):
    print(f"Reading image from: {input_image}")
    img = cv2.imread(input_image)

    if img is None:
        print(f"Error: Image {input_image} not found or unable to read.")
        return 

    flipped_img = cv2.flip(img, 1) 
    cv2.imwrite(output_image, flipped_img)
    print(f"Image flipped and saved at {output_image}")

input_folder = './Data/Images_Cleaned'
output_folder = './Data/Flip_Images'

os.makedirs(output_folder, exist_ok=True)

valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
for image_name in os.listdir(input_folder):
    if any(image_name.endswith(ext) for ext in valid_extensions):
        image_path = os.path.join(input_folder, image_name)
        output_path = os.path.join(output_folder, f"flipped_{image_name}")
        flip_image(image_path, output_path)


Reading image from: ./Data/Images_Cleaned\A-01.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-01.jpg
Reading image from: ./Data/Images_Cleaned\A-02.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-02.jpg
Reading image from: ./Data/Images_Cleaned\A-03.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-03.jpg
Reading image from: ./Data/Images_Cleaned\A-04.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-04.jpg
Reading image from: ./Data/Images_Cleaned\A-05.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-05.jpg
Reading image from: ./Data/Images_Cleaned\A-06.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-06.jpg
Reading image from: ./Data/Images_Cleaned\A-07.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-07.jpg
Reading image from: ./Data/Images_Cleaned\A-08.jpg
Image flipped and saved at ./Data/Flip_Images\flipped_A-08.jpg
Reading image from: ./Data/Images_Cleaned\A-09.jpg
Image flipped and saved at ./Data/Fli

KeyboardInterrupt: 